In [1]:
import numpy as np
import pickle
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
import shap
import pandas
from sklearn.metrics import pairwise_distances


In [2]:
glove_file = datapath('glove.6B.300d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")

_ = glove2word2vec(glove_file, tmp_file)

glove_model = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)


In [3]:
def nidxv(v, model):
    return np.array(model[np.array(model.similar_by_vector(v, topn=2))[:,0]]).sum(0) - 2*v

In [11]:
def sim_vec(vecs):
    res = np.zeros(len(vecs))
    i = 0
    for v in vecs:
        res[i] = abs(nidxv(v, glove_model).sum(axis = 0))
        i = i+1
    return res
    

In [12]:
xw = glove_model['man']
lr = nidxv(xw, glove_model)
print(lr)


[-2.20369995e-01 -5.54001331e-03 -2.66800016e-01  9.61900055e-02
  3.09948996e-02 -4.22100008e-01 -2.73175031e-01 -4.26419318e-01
 -8.15100074e-02 -1.25000477e-02 -1.55110002e-01  6.27000332e-02
  5.03919244e-01  4.30499762e-03  1.79654986e-01 -6.95829988e-02
  1.79450005e-01  2.09000111e-02 -2.65860021e-01 -3.34493011e-01
 -5.45569003e-01 -1.55920029e-01 -3.87559980e-02  3.44499946e-02
 -7.50899911e-02 -1.70865998e-01  5.42200208e-02 -1.25190005e-01
  1.57855093e-01  2.45499909e-02 -4.56589967e-01 -2.63530016e-01
 -2.11340010e-01 -2.31538981e-01 -1.21590018e-01  3.41670036e-01
 -6.50889993e-01 -7.77000189e-03  4.14460003e-01  1.11220002e-01
  2.55929947e-01 -1.21689975e-01 -6.58600032e-03  1.99730024e-02
  8.37400556e-03 -1.06849998e-01 -5.18800020e-02  4.28120017e-01
 -5.28299809e-02 -2.16220021e-01  1.08267993e-01  2.57037997e-01
  4.58856016e-01 -5.01530051e-01 -6.07380033e-01 -4.67804015e-01
 -4.60599959e-02 -4.00810003e-01  1.87580049e-01  1.64700150e-02
 -2.73239017e-01 -3.25440

In [13]:
t = np.zeros((2, 300))
t[0] = glove_model['man']
t[1] = glove_model['woman']
sim_vec(t)

array([4.84541851, 4.85488092])

In [14]:
ts = 40001
j = 0
test = np.empty(ts,dtype=object)
for w in list(glove_model.vocab.keys())[:ts]:
    test[j] = glove_model[w]
    j = j + 1

In [15]:
features = np.stack(test[:])
df_features = pandas.DataFrame(features[0:,0:], 
                      index=['Row '+str(i) for i in range(features.shape[0])], 
                      columns=['f'+str(i) for i in range(features.shape[1])]
                     )
df_features.head()



,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f290,f291,f292,f293,f294,f295,f296,f297,f298,f299
Row 0,0.046560,0.213180,-0.007436,-0.458540,-0.035639,0.236430,-0.288360,0.215210,-0.134860,-1.6413,...,-0.013064,-0.296860,-0.079913,0.195000,0.031549,0.285060,-0.087461,0.009061,-0.209890,0.053913
Row 1,-0.255390,-0.257230,0.131690,-0.042688,0.218170,-0.022702,-0.178540,0.107560,0.058936,-1.3854,...,0.075968,-0.014359,-0.073794,0.221760,0.146520,0.566860,0.053307,-0.232900,-0.122260,0.354990
Row 2,-0.125590,0.013630,0.103060,-0.101230,0.098128,0.136270,-0.107210,0.236970,0.328700,-1.6785,...,0.060148,-0.156190,-0.119490,0.234450,0.081367,0.246180,-0.152420,-0.342240,-0.022394,0.136840
Row 3,-0.076947,-0.021211,0.212710,-0.722320,-0.139880,-0.122340,-0.175210,0.121370,-0.070866,-1.5721,...,-0.366730,-0.386030,0.302900,0.015747,0.340360,0.478410,0.068617,0.183510,-0.291830,-0.046533
Row 4,-0.257560,-0.057132,-0.671900,-0.380820,-0.364210,-0.082155,-0.010955,-0.082047,0.460560,-1.8477,...,-0.012806,-0.597070,0.317340,-0.252670,0.543840,0.063007,-0.049795,-0.160430,0.046744,-0.070621


In [20]:
bckgr = shap.sample(df_features, 100)
explainer = shap.SamplingExplainer(sim_vec, bckgr)

In [17]:
x_test = df_features[:10]

In [21]:
shap_values = explainer.shap_values(x_test)


KeyboardInterrupt: 

In [ ]:
shap.summary_plot(shap_values, bckgr)

In [140]:
newweights = np.copy(glove_model.vectors)
newweights.shape

(400001, 50)

In [ ]:
#glove_model.vectors = weights
print(glove_model.most_similar(positive=['man']))
glove_model.save_word2vec_format('glove-vectors.txt', binary=False)

In [141]:

newweights = newweights[:, [30, 25, 26, 18, 2, 11, 49, 33]]

In [142]:
glove_model.vectors = newweights
print(glove_model.most_similar(positive=['man']))
glove_model.save_word2vec_format('shap-reduced-diff-8-vectors.txt', binary=False)

[('boy', 0.9864449501037598), ("'s", 0.9848877787590027), ('blow', 0.9708280563354492), ('war', 0.9698127508163452), ('ear', 0.9680918455123901), ('head', 0.9666916131973267), ('youngest', 0.9665636420249939), ('dubbed', 0.9646068811416626), ('bend', 0.9644647836685181), ('throat', 0.9636381268501282)]
